# Introduction to Text Mining

In this lab we look at a range of text mining techniques which are available as part of Scikit-learn.

In [29]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity

## Tokenizing Text

As our sample corpus of text, we will read a small collection of news articles. These articles have been stored in a single file and formatted so that one article appears on each line in the file.

In [30]:
fin = open("news-articles.txt", "r")
raw_documents = fin.readlines()
fin.close()
print("Read %d raw text documents" % len(raw_documents))

Read 45 raw text documents


Raw text documents are textual, not numeric. The first step in analysing unstructured documents is to split the raw text into individual tokens, each corresponding to a single term (word). As an example:

In [31]:
doc1 = raw_documents[0]
# print a snippet
print(doc1[0:300])

Parry puts Gerrard 'above money' Listen to the full interview on Sport on Five and the BBC Sport website from 1900 GMT. But Parry, speaking exclusively to BBC Sport, also admits Gerrard, who has been constantly linked with Chelsea, will have the final say on his future. He told BBC Five Live: "Steve


We will use the built-in scikit-learn tokenizer to split this document into tokens. Note that we will perform *case conversion* first to convert the entire text to lowercase.

In [32]:
tokenize = CountVectorizer().build_tokenizer()
# convert to lowercase, then tokenize
tokens1 = tokenize(doc1.lower())
print(tokens1)

['parry', 'puts', 'gerrard', 'above', 'money', 'listen', 'to', 'the', 'full', 'interview', 'on', 'sport', 'on', 'five', 'and', 'the', 'bbc', 'sport', 'website', 'from', '1900', 'gmt', 'but', 'parry', 'speaking', 'exclusively', 'to', 'bbc', 'sport', 'also', 'admits', 'gerrard', 'who', 'has', 'been', 'constantly', 'linked', 'with', 'chelsea', 'will', 'have', 'the', 'final', 'say', 'on', 'his', 'future', 'he', 'told', 'bbc', 'five', 'live', 'steven', 'is', 'above', 'money', 'he', 'is', 'the', 'future', 'of', 'liverpool', 'it', 'doesn', 'matter', 'if', 'it', '30m', '40m', 'or', '50m', 'we', 'will', 'not', 'accept', 'offers', 'but', 'we', 'are', 'also', 'realistic', 'enough', 'to', 'know', 'we', 'can', 'keep', 'steven', 'against', 'his', 'will', 'on', 'the', 'subject', 'of', 'liverpool', 'finances', 'parry', 'also', 'revealed', 'the', 'club', 'is', 'ready', 'to', 'explore', 'the', 'possibility', 'of', 'sponsorship', 'deal', 'for', 'its', 'proposed', 'new', 'stadium', 'and', 'responding', 't

We immediately see that many of the words here are not useful (e.g. "to", "the" etc.). Scikit-learn provides a list of such *stop words*:

In [33]:
stopwords = text.ENGLISH_STOP_WORDS
print(stopwords)

frozenset({'eleven', 'do', 'mostly', 'often', 'there', 'name', 'some', 'five', 'thin', 'system', 'may', 'thereby', 'each', 'yours', 'hereupon', 'whence', 'sixty', 'who', 'mine', 'at', 'whatever', 'nevertheless', 'for', 'somewhere', 'cant', 'once', 'couldnt', 'will', 'though', 'towards', 'nowhere', 'by', 'together', 'yet', 'beyond', 'well', 'thereafter', 'detail', 're', 'his', 'back', 'along', 'go', 'enough', 'might', 'amongst', 'since', 'whom', 'become', 'would', 'than', 'perhaps', 'everywhere', 'are', 'themselves', 'whereafter', 'whither', 'beside', 'whose', 'inc', 'noone', 'get', 'same', 'cry', 'him', 'becomes', 'least', 'they', 'anyone', 'someone', 'latterly', 'against', 'nor', 'bill', 'off', 'ten', 'several', 'nine', 'this', 'from', 'un', 'within', 'while', 'around', 'ours', 'rather', 'ourselves', 'have', 'how', 'four', 'everything', 'although', 'found', 'until', 'part', 'not', 'it', 'beforehand', 'be', 'two', 'down', 'top', 'hence', 'should', 'to', 'ltd', 'twenty', 'elsewhere', 'm

We can filter out these stopwords from our document:

In [34]:
filtered_tokens1 = []
for token in tokens1:
    if not token in stopwords:
        filtered_tokens1.append(token)
print(filtered_tokens1)

['parry', 'puts', 'gerrard', 'money', 'listen', 'interview', 'sport', 'bbc', 'sport', 'website', '1900', 'gmt', 'parry', 'speaking', 'exclusively', 'bbc', 'sport', 'admits', 'gerrard', 'constantly', 'linked', 'chelsea', 'final', 'say', 'future', 'told', 'bbc', 'live', 'steven', 'money', 'future', 'liverpool', 'doesn', 'matter', '30m', '40m', '50m', 'accept', 'offers', 'realistic', 'know', 'steven', 'subject', 'liverpool', 'finances', 'parry', 'revealed', 'club', 'ready', 'explore', 'possibility', 'sponsorship', 'deal', 'proposed', 'new', 'stadium', 'responding', 'criticism', 'bbc', 'sport', 'pundit', 'liverpool', 'stalwart', 'alan', 'hansen', 'insisted', 'talks', 'new', 'investment', 'ongoing', 'added', 'door', 'closed', 'shareholder', 'lifelong', 'fan', 'steve', 'morgan', 'parry', 'joined', 'liverpool', 'chief', 'executive', 'july', '1998', 'similar', 'role', 'premier', 'league', 'highs', 'lows', 'time', 'charge', 'anfield', 'busy', 'summer', 'overseeing', 'arrival', 'new', 'manager',

We will repeat this process for all documents:

In [35]:
all_filtered_tokens = []
for doc in raw_documents:
    # tokenize the next document
    tokens = tokenize(doc.lower())
    # remove the stopwords
    filtered_tokens = []
    for token in tokens:
        if not token in stopwords:
            filtered_tokens.append(token)  
    # add to the overall list
    all_filtered_tokens.append( filtered_tokens )
print("Created %d filtered token lists" % len(all_filtered_tokens) )

Created 45 filtered token lists


### Counting Tokens

A simple type of analysis that we might do is to count the number of times specific terms (words) appear in our corpus. We could do this by creating a dictionary of term frequency counts:

In [36]:
counts = Counter()
# process filtered tokens for each document
for doc_tokens in all_filtered_tokens:
    for token in doc_tokens:
        counts[token] += 1
print("Found %d unique terms in this corpus" % len(counts))

Found 3063 unique terms in this corpus


We would like to find the terms in the dictionary with the highest counts. The Python *Counter* collection makes this easy:

In [37]:
sorted_counts = counts.most_common()

The above creates a list of tuple pairs, where the first value is the key (i.e. the term) and the second value is the value (i.e the count). Let's display the top 20 terms.

In [38]:
for i in range(20):
    term = sorted_counts[i][0]
    count = sorted_counts[i][1]
    print( "%s (count=%d)" % (term, count)  )

yukos (count=118)
said (count=106)
oil (count=97)
liverpool (count=83)
gazprom (count=60)
russian (count=56)
win (count=50)
deal (count=42)
gerrard (count=41)
chelsea (count=41)
russia (count=40)
league (count=39)
auction (count=39)
rosneft (count=37)
company (count=37)
firm (count=37)
court (count=37)
yugansk (count=35)
bankruptcy (count=35)
year (count=34)


### Bag-of-Words Representation

In the *bag-of-words model*, each document is represented by a vector in a *m*-dimensional coordinate space, where *m* is number of unique terms across all documents. This set of terms is called the corpus *vocabulary*. Note that the positioning (context) of terms within the original document is lost in this model.

Since each document can be represented as a term vector, we can stack these vectors to create a full *document-term matrix*. We can easily create this matrix from a list of document strings using Scikit-learn:

In [39]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(raw_documents)
print(X.shape)

(45, 3288)


This process also build a vocabulary for the corpus, both in the form of a list and in the form of a dictionary:

In [40]:
terms = vectorizer.get_feature_names_out()
vocab = vectorizer.vocabulary_
print("Vocabulary has %d distinct terms" % len(terms))

Vocabulary has 3288 distinct terms


Display some sample terms:

In [41]:
print(terms[500:530])

['bristol' 'british' 'broke' 'brokerage' 'brom' 'brookings' 'brought'
 'bruce' 'brunswick' 'brutal' 'bryan' 'buenos' 'build' 'building' 'built'
 'bulgaria' 'bulgarian' 'burgas' 'buried' 'burnley' 'burren' 'business'
 'businesses' 'bust' 'busy' 'but' 'butler' 'buy' 'buyer' 'buying']


Since each column in the document-term matrix correspond to a term, we can look up the column associated with each term using the dictionary:

In [42]:
# what column is the term 'football' on?
vocab["football"]

1217

In [43]:
# what column is the term 'world' on?
vocab["world"]

3246

We can use the same Scikit-learn functionality to create a document-term matrix with N-grams. We specify an extra parameter ngram_range which specifies the shortest and longest token sequences to include. Length 1 is just a single token.

For instance, transform our input documents into a matrix, extracting single tokens and bigrams:

In [44]:
vectorizer = CountVectorizer(ngram_range=(1,2))
X = vectorizer.fit_transform(raw_documents)

Note the vocabulary is much larger now:

In [45]:
terms = vectorizer.get_feature_names_out()
vocab = vectorizer.vocabulary_
print("Vocabulary has %d distinct terms" % len(terms))

Vocabulary has 16075 distinct terms


Display some sample terms. Note that we see a mix of single tokens and bigrams (i.e. phrases of length 2):

In [46]:
print(terms[510:520])

['admitted victory' 'adriatic' 'adriatic coast' 'advanced'
 'advanced position' 'advantage' 'advantage and' 'advantage of' 'adverts'
 'adverts for']


## Text Preprocessing

A range of steps can be used to process text input files to reduce the number of terms used to represent the text and to improve the resulting bag-of-words model. These include:
- Minimum term length: Exclude terms of length < 2. Scikit-learn does this by default.
- Case conversion: Converting all terms to lowercase. Scikit-learn does this by default.
- Stop-word filtering: Remove terms that appear on a pre-defined "blacklist" of terms that are highly frequent and do- not convey useful information.
- Low frequency filtering: Remove terms that appear in very few documents.
- Stemming: Reduce words to their stems (or base forms).

Scikit-learn allows us to perform one or more of these steps by adapting the CountVectorizer.

We can use the built-in list of stop-words for a given language by just specifying the name of the language (lower-case):

In [47]:
vectorizer = CountVectorizer(stop_words="english")
X = vectorizer.fit_transform(raw_documents)
# Are standard stopwords gone?
"and" in vectorizer.vocabulary_

False

Or we could use our own custom stop-word list, which might be more appropriate for specific applications:

In [48]:
custom_stop_words = [ "and", "the", "game" ] 
vectorizer = CountVectorizer(stop_words=custom_stop_words)
X = vectorizer.fit_transform(raw_documents)
# Are custom stopwords gone?
"game" in vectorizer.vocabulary_

False

We can remove low frequency terms that appear in fewer than a specified number of documents:

In [49]:
# how many terms did we have with the last approach?
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(raw_documents)
print("Number of terms in model is %d" % len(vectorizer.vocabulary_) )

Number of terms in model is 3288


In [50]:
# build another matrix, but filter terms appearing in less than 5 documents
vectorizer = CountVectorizer(min_df = 5)
X = vectorizer.fit_transform(raw_documents)
print("Number of terms in model is %d" % len(vectorizer.vocabulary_) )

Number of terms in model is 473


To stem tokens to their base form, we need to use functionality from another third party library: **NLTK**. You may need to install this package manually, either through the Conda interface or via the Conda command line tool, using:
    
    conda install nltk

We can test out the standard English stemming algorithm (called the Porter Stemmer):

In [51]:
# import the standard English stemming algorithm
from nltk.stem.porter import PorterStemmer
words = ['plotted', 'flies', 'denied', 'sales', 'seas', 'computing', 'computed']
# try stemming each sample word
stemmer = PorterStemmer()
for w in words:
    print( stemmer.stem(w) )

plot
fli
deni
sale
sea
comput
comput


To use NLTK stemming with Scikit-learn, we need to create a custom tokenisation function:

In [52]:
# define the function
def stem_tokenizer(text):
    # use the standard scikit-learn tokenizer first
    standard_tokenizer = CountVectorizer().build_tokenizer()
    tokens = standard_tokenizer(text)
    # then use NLTK to perform stemming on each token
    stemmer = PorterStemmer()
    stems = []
    for token in tokens:
        stems.append(stemmer.stem(token))
    return stems

Now we can use our custom tokenizer with the standard CountVectorizer approach:

In [53]:
vectorizer = CountVectorizer(tokenizer=stem_tokenizer)
X = vectorizer.fit_transform(raw_documents)
# display some sample terms
terms = vectorizer.get_feature_names_out()
print(terms[200:220])

['alex' 'alexand' 'alexei' 'all' 'alleg' 'allen' 'allow' 'almost' 'alon'
 'along' 'alongsid' 'alonso' 'alreadi' 'also' 'altern' 'although'
 'alvalad' 'alway' 'am' 'ambit']


We can perform lemmatisation in the same way, using NLTK with Sckit-learn:

In [54]:
# import the WordNet lemmatizer
from nltk.stem import WordNetLemmatizer
# define the function
def lemma_tokenizer(text):
    # use the standard scikit-learn tokenizer first
    standard_tokenizer = CountVectorizer().build_tokenizer()
    tokens = standard_tokenizer(text)
    # then use NLTK to perform lemmatisation on each token
    lemmatizer = WordNetLemmatizer()
    lemma_tokens = []
    for token in tokens:
        lemma_tokens.append(lemmatizer.lemmatize(token))
    return lemma_tokens

Again we can use our custom tokenizer with the standard CountVectorizer approach. The output terms are somewhat easier to intrepret than those produced by stemming:

In [55]:
vectorizer = CountVectorizer(tokenizer=lemma_tokenizer)
X = vectorizer.fit_transform(raw_documents)
# display some sample terms
print(list(vectorizer.vocabulary_.keys())[0:35])

['parry', 'put', 'gerrard', 'above', 'money', 'listen', 'to', 'the', 'full', 'interview', 'on', 'sport', 'five', 'and', 'bbc', 'website', 'from', '1900', 'gmt', 'but', 'speaking', 'exclusively', 'also', 'admits', 'who', 'ha', 'been', 'constantly', 'linked', 'with', 'chelsea', 'will', 'have', 'final', 'say']


Let's put all of these steps together. Note that if we are applying lemmatization to our texts, we should also apply it to our stopwords:

In [56]:
# apply lemmatization to our previous English stopwords
lemma_stopwords = []
lemmatizer = WordNetLemmatizer()
for stopword in stopwords:
     lemma_stopwords.append(lemmatizer.lemmatize(stopword))

In [57]:
# now apply the vectorization processs
vectorizer = CountVectorizer(stop_words=lemma_stopwords, min_df=3, tokenizer=lemma_tokenizer)
X = vectorizer.fit_transform(raw_documents)
print(X.shape)

(45, 678)


In [58]:
print(list(vectorizer.vocabulary_.keys())[0:35])

['gerrard', 'money', 'bbc', 'gmt', 'speaking', 'linked', 'chelsea', 'final', 'say', 'future', 'told', 'live', 'steven', 'liverpool', 'doesn', 'matter', 'offer', 'know', 'finance', 'revealed', 'club', 'ready', 'deal', 'new', 'stadium', 'alan', 'insisted', 'talk', 'investment', 'added', 'closed', 'shareholder', 'fan', 'steve', 'chief']


## Term Weighting

As well as including/excluding terms, we can also modify or weight the frequency values themselves. We can improve the usefulness of the document-term matrix by giving more weight to the more "important" terms.

The most common normalisation is *term frequency–inverse document frequency* (TF-IDF). In Scikit-learn, we can generate at TF-IDF weighted document-term matrix by using TfidfVectorizer() in place of CountVectorizer().

In [59]:
# we can pass in the same preprocessing parameters
vectorizer = TfidfVectorizer(stop_words="english",min_df = 5)
X = vectorizer.fit_transform(raw_documents)
# display some sample weighted values
print(X[0])

  (0, 130)	0.3974597791748038
  (0, 204)	0.21755328169056634
  (0, 30)	0.13831731070303818
  (0, 54)	0.088438210466365
  (0, 111)	0.034579327675759546
  (0, 261)	0.1423631472946755
  (0, 124)	0.12709054744517082
  (0, 301)	0.05019683116727541
  (0, 183)	0.03819778147145201
  (0, 283)	0.34167155350722117
  (0, 184)	0.27655952066779993
  (0, 167)	0.088438210466365
  (0, 59)	0.2277810356714808
  (0, 243)	0.03819778147145201
  (0, 77)	0.21338170139457197
  (0, 211)	0.1927835160936817
  (0, 158)	0.0992935718534424
  (0, 9)	0.02586094969309587
  (0, 282)	0.21755328169056634
  (0, 55)	0.0305738291672926
  (0, 101)	0.03625888028176106
  (0, 253)	0.034579327675759546
  (0, 177)	0.04739884783718552
  (0, 298)	0.07529524675091312
  (0, 19)	0.0953179105838781
  :	:
  (0, 78)	0.034579327675759546
  (0, 289)	0.03819778147145201
  (0, 326)	0.023054258878629028
  (0, 49)	0.07639556294290402
  (0, 195)	0.06915865535151909
  (0, 245)	0.033097857284480794
  (0, 87)	0.0305738291672926
  (0, 83)	0.03457932

## Measuring Similarity

*Cosine similarity*: the most common approach for measuring similarity between two documents in a bag-of-words representation is to look at the cosine of the angle between their corresponding two term vectors. The motivation is that vectors for documents containing similar terms will point in the same direction in the $m$-dimensional vector space.

As an example, let's find the most similar document to the first document in our collection.

In [60]:
# first document - just display the start of it
print(raw_documents[0][0:300])

Parry puts Gerrard 'above money' Listen to the full interview on Sport on Five and the BBC Sport website from 1900 GMT. But Parry, speaking exclusively to BBC Sport, also admits Gerrard, who has been constantly linked with Chelsea, will have the final say on his future. He told BBC Five Live: "Steve


In [61]:
# measure the cosine similarity between the first document vector and all of the other document vectors
max_cos = 0
best_row = 0
for row in range(1,X.shape[0]):
    cos = cosine_similarity(X[0], X[row])
    # best so far?
    if cos > max_cos:
        max_cos = cos
        best_row = row

In [62]:
print("Most similar document was row %d: cosine similarity = %.3f" % (best_row, max_cos) )

Most similar document was row 16: cosine similarity = 0.550


/var/folders/4j/3w98pmcj1qb9m2ysc0djsvnc0000gn/T/ipykernel_14474/2668773183.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("Most similar document was row %d: cosine similarity = %.3f" % (best_row, max_cos) )


In [63]:
# most similar document - just display the start of it
print(raw_documents[best_row][0:300])

Liverpool pledge to keep Gerrard Liverpool chief executive Rick Parry insists the club will never sell Steven Gerrard amid reports Chelsea will renew their bid to lure him from Anfield. Gerrard reiterated his desire to win trophies with the Reds after his superb Champions League winner on Wednesday.
